# Breve Introdução

**USE PYSPARK OU SQL PARA RESOLVER AS QUESTÕES ABAIXO**


**NENHUM DADO DISPONIBILIZADO É REAL**

Uma das áreas de negócio da XP é o B2B, onde empresas denominadas de escritórios de investimentos, fazem uma parceria com a XP para que seus clientes invistam através de nossa plataforma. Esses escritórios atendem nossos clientes com os assessores autônomos de invesmento, que ajudam as pessoas a investirem melhor. Depois de uma reunião com um dos gerentes da nossa área B2B, ficou claro que a carteira de muitos clientes estava desalinhada com as carteiras recomendadas para seu perfil e caso conseguíssemos reduzir esse desalinhamento, iriamos ajudar os clientes tornando suas carteiras mais seguras e rentáveis ao mesmo tempo que aumentaríamos nossa receita. Criamos então um projeto separado em 4 etapas para conseguirmos aumentar a aderência das carteiras de invesmento dos clientes dentro de seu suitability.


**Entendendo melhor a nosso banco de dados**

As tabelas disponibilizadas para esse desafio como você pode perceber na query acima possuem dois prefixos diferentes, sendo eles; FAT_ e DIM_

* **FAT_**: São as tabelas fato, o que significa que são tabelas que guardam os fatos históricos de algum acontecimento. Então de forma periódica algumas informações podem se repetir
  
  Exemplo: Se estivéssemos falando de uma tabela fato de vendas por cliente, teríamos uma tabela onde cada linha seria o total de vendas de um cliente naquele dia, então o mesmo cliente poderá se repetir na tabela várias vezes, porque as vendas podem acontecer em vários dias diferentes.
  
* **DIM_**: São as tabelas dimensão, elas se relacionam com as tabelas fato e guardam informações sobre algo, geralmente são tabelas com menos linhas e mais colunas que as fato.

  Exemplo: Pegando o exemplo de vendas da fato explicada acima. Na fato a conta do cliente vai se repetir várias vezes por conta dos dias, então é mantido na tabela fato apenas o código do cliente e na tabela dimensão vão ter todas as informações sobre os clientes, por exemplo o nome, idade, profissão e etc... dessa forma essas informações de texto não vão se repetir na tabela fato tornando a tabela muito mais rápida e menor.
  
**O que são os COD_DIM?**

Você vai perceber que em nossas tabelas existem colunas com o nome COD_DIM_, essas colunas são códigos que são usados para se relacionar dimensões e fatos, então por exemplo a coluna COD_DIM_TEMPO da tabela DIM_TEMPO, também existe na tabela fato FAT_NET_MENSAL e é usada para levar informações sobre o tempo para a tabela fato, que por sua vez fica mais rápida por não ter dados de texto.

# Preparação (Desafio 0)

Após ter criado o cluster e carregado os arquivos CSV para o Databricks chegou a hora de transformá-los em tabelas.
Use a célula abaixo ou (crie células, se necessário) para transformar os arquivos fornecidos em Tabelas do Databricks.




Quando uma tabela é criada o resultado da query fica como "Query returned no results", não se preocupe, isso significa que a tabela foi criada com sucesso, você pode consulta-la após isso fazendo um select, como mostrado na célula a seguir

In [0]:
%python
lista_nomes_tabela = [
    'fat_recomendacao',
    'fat_net_mensal',
    'dim_produto',
    'dim_conta_suitability',
    'dim_canal_assessor'
]

for nome_tabela in lista_nomes_tabela:
#    < .... CÓDIGO AUXILIAR PARA CRIAÇÃO DAS TABELAS ....>
#    <.... COMPLETE COM AS LINHAS NECESSÁRIAS PARA CRIAR AS TABELAS A PARTIR DOS ARQUIVOS FORNECIDOS ....> 
    
    (
        df.write
          .mode('overwrite')
          .option("overwriteSchema", "true")
          .saveAsTable(nome_tabela)
    )


Para finalizar nossa preparação, essa etapa consiste em criar uma tabela que lista o suitability e a custódia mensal dos clientes do B2B para cada estratégia diferente de produto.
Portanto, é importante identificar qual (ou quais) tabela(s) fato ou dimensão você irá usar.

**Notas:**
- Suitability é o perfil do investidor, pode ser conservador, moderado ou agressivo e representa o quanto de risco o cliente está disposto a assumir pelo retorno
- Custódia é o tanto de dinheiro dos clientes que está investido dentro da plataforma XP. Esse dinheiro não é da XP
- Estratégia de produto é um tipo de classificação de produto que assume as seguintes categorias: Prefixado, Pós-Fixado, Inflação, Multimercado, Renda Variável e Internacional
- Se o assessor de algum cliente é do B2B, então esse cliente também é considerado desse canal


*Lembre-se de antes de usar esse notebook ativar o cluster, para isso uma vez que ele já tenha sido criado, basta clicar no botão "Detached" encontrado no canto esquerdo superior abaixo do título do notebook. Para mais informações olhe o tutorial em word*

Agora, com as tabelas tabelas fato e dimensão criadas crie a *fat_custodia_conta_mensal*:

##### Atenção: Nomeie essa tabela com o seguinte nome: fat_custodia_conta_mensal

# Desafio 1

Usando a tabela gerada na primeira etapa com o nome de *fat_custodia_conta_mensal*, faça um agrupamento de tal forma que 1) exiba a custódia mais recente do cliente independente da estratégia e 2) em seguida crie uma coluna que exiba a alocação em cada estratégia de produto para cada cliente.

Alocação: % de custódia que o produto tem na carteira

Layout de resultado esperado:

| COD_DIM_CONTA | SUITABILITY | ESTRATEGIA     | CUSTODIA  | ALOCACAO            |
|---------------|-------------|----------------|-----------|---------------------|
| 13            | Moderado    | Internacional  | 8478.79   | 0.046 |
| 13            | Moderado    | Pós-Fixado     | 116706.06 | 0.641  |
| 13            | Moderado    | Renda Variável | 56791.49  | 0.312   |
| 14            | Moderado    | Internacional  | 848.92   | 0.046 |
| 14            | Moderado    | Pós-Fixado     | 116.61 | 0.641  |
| 14            | Moderado    | Renda Variável | 791.44  | 0.312   |

# Desafio 2

Calcule a diferença de alocação em cada estratégia de produto baseado na alocação recomendada pela tabela *FAT_RECOMENDACAO*. Essa nova coluna deve mostrar a diferença em pontos percentuais (valor absoluto) entre a carteira do cliente e a carteira recomendada. Após isso filtre os 5 clientes com maior diferença de alocação.

**Notas:**
- Para filtrar os 5 clientes com maior diferença de alocação é preciso descobrir a diferença de alocação total por cliente (desconsiderando a quebra de estratégia).
- Adote como premissa que vamos usar a recomendação mais recente da fat_recomendacao, você pode fazer isso filtrando a data máxima

Nessa etapa você precisará usar o resultado do exercício anterior. 

Além de usar o resultado do exercício anterior, use a tabela fat_recomendacao disponibilizada para chegar ao resultado esperado, lembrando que a recomendação é feita para cada classe de ativo de acordo com o suitability e isso pode mudar todos os meses


Layout de resultado esperado:

| COD_DIM_CONTA | SUITABILITY | ESTRATEGIA     | CUSTODIA | ALOCACAO            | ALOCACAO_RECOMENDADA | DESVIO_ALOCACAO     |
|---------------|-------------|----------------|----------|---------------------|----------------------|---------------------|
| 40            | Conservador | Internacional  | 51676.24 | 0.207 | 0.1                  | 0.107 |
| 40            | Conservador | Pós-Fixado     | 81623.5  | 0.328 | 0.9                  | 0.571  |
| 40            | Conservador | Renda Variável | 115475.6 | 0.464  | 0                   | 0.464  |

# Desafio 3

Bem agora que você construíu os dados de alocação recomendada e sabe melhor como está a carteira dos clientes, quais seriam os próximos passos? Sinta-se a vontade para usar todas as tabelas disponibilizadas até então.

Além disso o time de produtos te disponibilizou uma tabela com o ROA médio para cada estratégia de produto. Essa tabela pode ser acessada com o nome de *TB_ROA_MEDIO*

**Notas:**
- ROA = Revenue Over Assets = Receita / Custódia
- Receita = Dinheiro que a XP ganha com taxas, ou operações do cliente
- Custódia = Dinheiro dos clientes que está investido dentro da plataforma XP

**Sugestões de evolução:**

Sinta-se à vontade para fazer qualquer uma das sugestões ou ainda qualquer outra ideia que você tiver, use a criatividade e mostre seus conhecimentos técnicos, lembre-se que aqui no databricks você pode usar não só o SQL mas também o Python ou R caso você tenha conhecimento sob essas linguagens. Se achar válido explore seus conhecimentos em estatística, Pandas, Sklearn ou outros.

Algumas sugestões:
- Calcular a receita potencial gerada com a alocação recomendada dos clientes, ou seja, hoje os clientes possuem um desvio para a alocação recomendada, se esses clientes vendessem seus ativos e comprassem outros quanto de receita ele geraria para a XP? Você pode usar a tabela de ROA médio por estratégia de produto para fazer esse cálculo
- Analisar quais eram os clientes com maior custódia total por suitability
- Analisar os produtos com maior alocação dos clientes dentro do canal B2C
- Analisar o histórico de distribuição de produtos do cliente com os maiores desvios atualmente
- Exibir alguns gráficos usando o databricks ou o python
- Fazer uma projeção de custódia total do canal B2B e B2C